In [1]:
"""This code calculated to use a high res. topo tiff, and add storm surge water levels + SLR
to get flood depths in a tiff. 
flood depth = Total water level - topo 
Written by Taran Kalra, August 19, 2024"""


'This code calculated to use a high res. topo tiff, and add storm surge water levels + SLR\nto get flood depths in a tiff. \nflood depth = Total water level - topo \nWritten by Taran Kalra, August 19, 2024'

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat              
import rioxarray
import rasterio 
#from create_raster import make_swan_raster_StructuredGrid

In [3]:
cases = ["1perc", "2perc", "5perc", "10perc", "20perc", "50perc"]
years = ["100yr", "50yr", "20year", "10year", "5year", "2year"] # year of return 
TWL = [    7.57,   6.69,    5.69,     5.03,     4.44,    3.77] # Extracted from transect 7 
SLR  = 0.76 # Sea level rise in 50 years from the report (this will take us to 2070) # Not using SLR 

In [4]:
scene = "Degraded" # "Restored"

if (scene == "Restored"):
    scenario = "Restored" # "Degraded"
    scenario1= "restored" # degraded"  
    scenario_out = "alt"  # "no_alt"
    #gridfile = r'C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\Grids\restored_basket_flats.tif'
    gridfile = r'C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\Grids\grid_clipped_DEM_Maurice_river1.tif'
else:
    scenario = "Degraded"
    scenario1 = "degraded"
    scenario_out = "no_alt"
    #gridfile = r'C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\Grids\degraded_basket_flats.tif'
    gridfile = r'C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\Grids\grid_clipped_DEM_Maurice_river1.tif'

# input file names
#input_directory = r'C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\Grids'
output_directory = r'C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\final\Flood_depths'

In [5]:
print(TWL)

[7.57, 6.69, 5.69, 5.03, 4.44, 3.77]


In [6]:
# Loop over the filenames in the directory
for i, case in enumerate(cases): 
    outfile = f'{output_directory}\{case}.tif' 
 
    print(gridfile) 
        
    # Grid file path specified 
    # this is the finer TIFF file 
    finer_xds = rioxarray.open_rasterio(gridfile) 
    topobathy = finer_xds.squeeze() 

    # temp
    ds_new    = topobathy
    topobathy = np.where(topobathy<-9999, np.nan, topobathy)

    # Need this to keep the object type correct 
    ds_new = ds_new*0.0
    
    # Fill the array to get flood depth, note TWL is variable for all scenarios
    # ds_new is the wave setup interpolated the SWAN grid that is refined 
    ds_new = ds_new - topobathy  + TWL[i]   # topobathy is in meter from NOAA has reverse sign (ocean is negative, land is positive) 
    print(type(ds_new))
    print(outfile)
   
  
    ds_new.rio.to_raster(outfile) # save the new array in a raster before changing the profile of the raster

    print(np.nanmin(ds_new))
    print(np.nanmax(ds_new)) 

C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\Grids\grid_clipped_DEM_Maurice_river1.tif
<class 'xarray.core.dataarray.DataArray'>
C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\final\Flood_depths\1perc.tif
-22.558971
51.531597
C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\Grids\grid_clipped_DEM_Maurice_river1.tif
<class 'xarray.core.dataarray.DataArray'>
C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\final\Flood_depths\2perc.tif
-23.43897
50.651596
C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\Grids\grid_clipped_DEM_Maurice_river1.tif
<class 'xarray.core.dataarray.DataArray'>
C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\final\Flood_depths\5perc.tif
-24.43897
49.651596
C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\Grids\grid_clipped_DEM_Maurice_river1.tif
<class 'xarray.core.dataarray.DataArray'>
C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\final\Flood_depths\10perc.tif
-25.09897
48.991596
C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\Grids\grid_clipped_DEM_Maurice_river1.tif
<class 'xa

In [7]:

#fig = plt.figure(figsize=(15,10))
#ax = fig.add_subplot(1,1,1)
#z1_plot = ax.pcolormesh(fine_data.x.values, fine_data.y.values, array, cmap='viridis', vmin=-20, vmax=20)
#plt.colorbar(z1_plot)